In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [6]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312      1.0 -9.077911e-06                  NaN       NaN
0.000312      NaN           NaN                  1.0 -0.000020
0.000750      2.0 -9.044177e-06                  2.0 -0.000042
0.001052      3.0 -8.977873e-06                  3.0 -0.000008
0.001476      4.0 -8.891137e-06                  4.0 -0.000008
0.002070      5.0 -8.802642e-06                  5.0 -0.000007
0.002904      6.0 -8.701331e-06                  6.0 -0.000007
0.004074      7.0 -8.587891e-06                  7.0 -0.000007
0.005714      8.0 -8.467483e-06                  8.0 -0.000007
0.008015      9.0 -8.334229e-06                  9.0 -0.000007
0.011243     10.0 -8.189517e-06                 10.0 -0.000007
0.015771     11.0 -8.033162e-06                 11.0 -0.000007
0.022122     12.0 -7.864161e-06                 12.0 -0.000007
0.031031     13.0 -7.681932e-06                 13.0 -0.000008
0.043528     14.0 -7.429187e-06                 14.0 -0.000012
0.061057     15.0 -6.352139e-06                 15.0 -0.000012
0.085645     16.0 -3.143961e-06                 16.0 -0.000008
0.120136     17.0  3.070876e-06                 17.0 -0.000005
0.168516     18.0  1.316197e-05                 18.0 -0.000001
0.236378     19.0  2.882620e-05                 19.0  0.000005
0.331549     20.0  5.243065e-05                 20.0  0.000014
0.465100     21.0  8.718533e-05                 21.0  0.000028
0.652400     22.0  1.317870e-04                 22.0  0.000048
0.915100     23.0  1.501447e-04                 23.0  0.000056
1.283650     24.0  1.322895e-04                 24.0  0.000048
1.800600     25.0  1.043212e-04                 25.0  0.000037
2.525700     26.0  7.352555e-05                 26.0  0.000024
3.542800     27.0  4.864666e-05                 27.0  0.000014
4.969550     28.0  3.025854e-05                 28.0  0.000006
6.970850     29.0  1.688221e-05                 29.0  0.000001
9.778100     30.0  7.295138e-06                 30.0 -0.000003
13.715850    31.0  5.647570e-07                 31.0 -0.000005
19.239350    32.0 -4.019626e-06                 32.0 -0.000007
26.987250    33.0 -7.044272e-06                 33.0 -0.000008
37.855300    34.0 -8.977668e-06                 34.0 -0.000008
53.100050    35.0 -1.019125e-05                 35.0 -0.000009
73.887500    36.0 -1.031378e-05                 36.0 -0.000009
97.662500    37.0 -1.005772e-05                 37.0 -0.000009
121.437500   38.0 -1.066284e-05                 38.0 -0.000009
145.212500   39.0 -1.178797e-05                 39.0 -0.000011
168.987500   40.0 -1.345305e-05                 40.0 -0.000013
192.762500   41.0 -1.895580e-05                 41.0 -0.000020
216.537500   42.0 -3.707884e-05                 42.0 -0.000046
240.312500   43.0 -5.249580e-05                 43.0 -0.000084
264.087500   44.0 -3.810979e-05                 44.0 -0.000126
287.862500   45.0  6.037806e-05                 45.0 -0.000141
311.637500   46.0  3.305724e-04                 46.0 -0.000071
335.412500   47.0  8.930704e-04                 47.0  0.000179
359.187500   48.0  1.869640e-03                 48.0  0.000731
382.962500   49.0  3.218443e-03                 49.0  0.001648
406.737500   50.0  5.118575e-03                 50.0  0.003071
430.512500   51.0  7.664960e-03                 51.0  0.005112
454.287500   52.0  1.093965e-02                 52.0  0.007937
478.062500   53.0  1.501085e-02                 53.0  0.011681
501.837500   54.0  1.989912e-02                 54.0  0.016420
525.612500   55.0  2.567105e-02                 55.0  0.022040
549.387500   56.0  3.232143e-02                 56.0  0.028668
573.162500   57.0  3.987959e-02                 57.0  0.036269
596.937500   58.0  4.840540e-02                 58.0  0.044802
620.712500   59.0  5.789584e-02                 59.0  0.054155
644.487500   60.0  6.843980e-02              

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -65.454986  0.000000e+00 -65.454986        NaN   
1.000000e-08 1            NaN           NaN        NaN -65.458622   
6.244000e-04 2     -65.454986  5.260138e-10 -65.454986 -65.458622   
8.759000e-04 3     -65.454986  7.414555e-10 -65.454986 -65.458622   
1.228600e-03 4     -65.454986  1.054060e-09 -65.454986 -65.458622   
1.723400e-03 5     -65.454986  1.508019e-09 -65.454986 -65.458622   
2.417400e-03 6     -65.454986  2.167135e-09 -65.454986 -65.458622   
3.390900e-03 7     -65.454986  3.124159e-09 -65.454986 -65.458622   
4.756500e-03 8     -65.454986  4.513637e-09 -65.454986 -65.458622   
6.672000e-03 9     -65.454986  6.530760e-09 -65.454986 -65.458622   
9.358900e-03 10    -65.454986  9.458924e-09 -65.454986 -65.458622   
1.312780e-02 11    -65.454986  1.370930e-08 -65.454986 -65.458622   
1.841450e-02 12    -65.454986  1.987869e-08 -65.454986 -65.458622   
2.583020e-02 13    -65.454986  2.883298e-08 -65.454986 -65.458622   
3.623230e-02 14    -65.454986  4.183025e-08 -65.454986 -65.458622   
5.082340e-02 15    -65.454986  6.087255e-08 -65.454986 -65.458622   
7.129060e-02 16    -65.454986  9.196099e-08 -65.454986 -65.458622   
1.000000e-01 17    -65.454986  1.506978e-07 -65.454986 -65.458622   
1.402710e-01 18    -65.454986  2.669429e-07 -65.454986 -65.458623   
1.967600e-01 19    -65.454986  4.964204e-07 -65.454986 -65.458623   
2.759970e-01 20    -65.454986  9.476689e-07 -65.454985 -65.458623   
3.871000e-01 21    -65.454986  1.831155e-06 -65.454985 -65.458623   
5.431000e-01 22    -65.454986  3.555890e-06 -65.454983 -65.458623   
7.617000e-01 23    -65.454986  6.798859e-06 -65.454980 -65.458624   
1.068500e+00 24    -65.454986  1.181466e-05 -65.454974 -65.458624   
1.498800e+00 25    -65.454986  1.821750e-05 -65.454967 -65.458624   
2.102400e+00 26    -65.454986  2.578379e-05 -65.454960 -65.458625   
2.949000e+00 27    -65.454987  3.413795e-05 -65.454952 -65.458627   
4.136600e+00 28    -65.454989  4.318214e-05 -65.454946 -65.458629   
5.802500e+00 29    -65.454993  5.294263e-05 -65.454940 -65.458632   
8.139200e+00 30    -65.454998  6.345100e-05 -65.454935 -65.458637   
1.141700e+01 31    -65.455007  7.474026e-05 -65.454932 -65.458643   
1.601470e+01 32    -65.455019  8.684349e-05 -65.454932 -65.458652   
2.246400e+01 33    -65.455035  9.981135e-05 -65.454935 -65.458664   
3.151050e+01 34    -65.455056  1.136860e-04 -65.454942 -65.458679   
4.420010e+01 35    -65.455084  1.283847e-04 -65.454956 -65.458699   
6.200000e+01 36    -65.455120  1.424350e-04 -65.454977 -65.458725   
8.577500e+01 37    -65.455161  1.539974e-04 -65.455007 -65.458757   
1.095500e+02 38    -65.455198  1.626842e-04 -65.455035 -65.458787   
1.333250e+02 39    -65.455238  1.734095e-04 -65.455065 -65.458821   
1.571000e+02 40    -65.455289  1.908696e-04 -65.455098 -65.458863   
1.808750e+02 41    -65.455356  2.201850e-04 -65.455136 -65.458918   
2.046500e+02 42    -65.455472  2.824903e-04 -65.455189 -65.459014   
2.284250e+02 43    -65.455773  4.789657e-04 -65.455294 -65.459272   
2.522000e+02 44    -65.456445  1.003094e-03 -65.455442 -65.459864   
2.759750e+02 45    -65.457777  2.228263e-03 -65.455549 -65.461059   
2.997500e+02 46    -65.460179  4.800476e-03 -65.455379 -65.463247   
3.235250e+02 47    -65.464186  9.737766e-03 -65.454448 -65.466950   
3.473000e+02 48    -65.470452  1.851952e-02 -65.451933 -65.472775   
3.710750e+02 49    -65.479623  3.295547e-02 -65.446667 -65.481372   
3.948500e+02 50    -65.491700  5.409713e-02 -65.437603 -65.492821   
4.186250e+02 51    -65.507187  8.399987e-02 -65.423188 -65.507611   
4.424000e+02 52    -65.526583  1.249823e-01 -65.401601 -65.526186   
4.661750e+02 53    -65.550368  1.795773e-01 -65.370791 -65.549095   
4.899500e+02 54    -65.579001  2.504854e-01 -65.328516 -65.576878   
5.137250e+02 55    -65.6

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')